# Appendix 1b. RMEstimator test with mock data - Bayesian evidence

**Author: [Hannu Parviainen](mailto:hpparvi@gmail.com)** </br>
**Last edited: 19 September 2023**


In [1]:
%matplotlib inline

from numba import njit
from numpy import array, arange
from numpy.random import uniform, seed
from tqdm.auto import tqdm

from dynesty import NestedSampler
from dynesty import plotting as dyplot

from spright import RMEstimator
from spright.mock import create_mock_sample
from spright.model import lnlikelihood_sample, map_pv

In [2]:
population_sizes = 50, 75, 100, 150, 200
ww_strenghts = 0.0, 0.5, 1.0
batches = arange(5)

In [3]:
pv = array([1.3,  # Rocky transition start 
             2.4,  # Puffy transition end
             0.0,  # WW population strength
             0.0,  # WW Population shape
             0.2,  # Rocky planet iron ratio
             0.5,  # WW water ratio
             3.0,  # SN density at r=2
            -0.9,  # SN density exponent
            -0.4,  # log10 RP density pdf scale
            -0.3,  # log10 WW density pdf scale
            -0.4   # log10 SN density pdf scale
           ])

In [4]:
h0 = array([[ 0.8,  1.8], 
             [ 1.8,  3.5], 
             [ 0.0,  0.01], 
             [-1.0,  1.0],
             [ 0.0,  0.5],
             [ 0.3,  0.7],
             [ 1.0,  4.0],
             [-2.5,  0.1],
             [-1.2,  0.5],
             [-1.2,  1.0],
             [-1.2,  0.5]])

h1 = h0.copy()
h1[2] = [0.05, 0.5]

h2 = h0.copy()
h2[2] = [0.5, 1.0]

@njit
def tr0(u):
    return h0[:,0] + u*(h0[:,1]-h0[:,0])

@njit
def tr1(u):
    return h1[:,0] + u*(h1[:,1]-h1[:,0])

@njit
def tr2(u):
    return h2[:,0] + u*(h2[:,1]-h2[:,0])

In [5]:
logz = {}
logze = {}

for psize in tqdm(population_sizes):
    for wws in tqdm(ww_strenghts):
        pv[2] = wws
        for ibatch in tqdm(batches):
            seed(ibatch)
            names = array(psize*[''])
            radius = uniform(0.5, 5.5, psize)
            mass = create_mock_sample(radius, pv, 'mass')
            radius_e = radius * uniform(0.01, 0.08, size=psize)
            mass_e = mass * uniform(0.03, 0.24, size=psize)
    
            rme = RMEstimator(nsamples=100, names=names, radii=(radius, radius_e), masses=(mass, mass_e))
            lpf = rme.lpf
            rdm = lpf.rdm
            
            def lnlikelihood(pv):
                return lnlikelihood_sample(pv, lpf.density_samples, lpf.radius_samples,
                                       rdm._r0, rdm._dr, rdm.drocky, rdm.dwater)
                
            s0 = NestedSampler(lnlikelihood, tr0, 11, nlive=1000)
            s1 = NestedSampler(lnlikelihood, tr1, 11, nlive=1000)
            s2 = NestedSampler(lnlikelihood, tr2, 11, nlive=1000)

            s0.run_nested(print_progress=False)
            s1.run_nested(print_progress=False)
            s2.run_nested(print_progress=False)

            logz[psize, int(100*wws), ibatch, 'H0'] = s0.results.logz[-1]
            logz[psize, int(100*wws), ibatch, 'H1'] = s1.results.logz[-1]
            logz[psize, int(100*wws), ibatch, 'H2'] = s2.results.logz[-1]

            logze[psize, int(100*wws), ibatch, 'H0'] = s0.results.logzerr[-1]
            logze[psize, int(100*wws), ibatch, 'H1'] = s1.results.logzerr[-1]
            logze[psize, int(100*wws), ibatch, 'H2'] = s2.results.logzerr[-1]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
logz

{(50, 0, 0, 'H0'): -51.740220620235185,
 (50, 0, 0, 'H1'): -52.6721081967136,
 (50, 0, 0, 'H2'): -54.45469372849778,
 (50, 0, 1, 'H0'): -59.41047254354563,
 (50, 0, 1, 'H1'): -59.32429271211719,
 (50, 0, 1, 'H2'): -59.9355578156648,
 (50, 0, 2, 'H0'): -53.97914238911576,
 (50, 0, 2, 'H1'): -55.69035795392055,
 (50, 0, 2, 'H2'): -59.39778240866274,
 (50, 0, 3, 'H0'): -54.74441091652819,
 (50, 0, 3, 'H1'): -56.8372452243977,
 (50, 0, 3, 'H2'): -61.474483431598635,
 (50, 0, 4, 'H0'): -45.27734778950429,
 (50, 0, 4, 'H1'): -45.78233422363854,
 (50, 0, 4, 'H2'): -47.22356117106534,
 (50, 50, 0, 'H0'): -50.0476827891481,
 (50, 50, 0, 'H1'): -49.64267287910525,
 (50, 50, 0, 'H2'): -50.095759098203615,
 (50, 50, 1, 'H0'): -57.270390149378564,
 (50, 50, 1, 'H1'): -56.38849266220588,
 (50, 50, 1, 'H2'): -56.30123820979275,
 (50, 50, 2, 'H0'): -57.923458364865446,
 (50, 50, 2, 'H1'): -56.73640215434681,
 (50, 50, 2, 'H2'): -57.60463512737293,
 (50, 50, 3, 'H0'): -47.99073761695592,
 (50, 50, 3, '

---

<center>
©2023 Hannu Parviainen
</center>